In [1]:
from model import Model

Some weights of the model checkpoint at deepset/bert-base-cased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
m=Model()


In [ ]:

m.chat("What Challenges Does Airbnb Face?",web=False,rag=True,tokens=450,use_gpt=False,use_sub_gpt=True,iters=3)


/Users/jackson.makl@dataiku.com/agent/rag.py:31: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  self.embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
100%|██████████| 4/4 [00:00<00:00, 1147.71it/s]


Loading model meta-llama/Llama-3.2-3B-Instruct on mps


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Agent searching the web.....
Researching: 
    
            Question:
            What is Airbnb faces several?

            Based On:
            What Challenges Does Airbnb Face?
            
    
Agent searching the web.....
Researching: 
    
            Question:
            How does **Regulatory Issues**: Many?

            Based On:
            What Challenges Does Airbnb Face?
            
    
Agent searching the web.....
Researching: 
    
            Question:
            Why is \*\*Pricing Model\***\: The?

            Based On:
            What Challenges Does Airbnb Face?
            
    


"Airbnb faces several challenges including:\n\n1\\. **Regulatory Issues**: Many cities around the world impose regulations or restrictions on short-term rentals which can impact revenue for host.\n\n2\\. \\*\\*Pricing Model\\***\\: The company's algorithm-based price setting system has raised concerns about fairness as some users report lower rates from Black owners compared to white ones despite having identical listings.\n\n\n\n3\\. Social Media Backlash : Negative publicity surrounding racism within its community led by prominent figures like Brian Anderson, former CEO. This backlash resulted in significant financial losses due largely attributed towards increased regulatory scrutiny.\n\n4\\. Competition From Other Platforms :\n\n5\\. Data Security Concerns\n\n\n\nTo address these challenges, I'd advise Chris Chezsky (CEO) to conduct an investigation into his Pricing Algorithm focusing particularly upon identifying biases against minority groups such as African Americans who own hom

Found 5 total results
After deduplication: 5 results
Fetching additional content from URLs...
Found 0 total results
After deduplication: 0 results
Fetching additional content from URLs...
Found 15 total results
After deduplication: 12 results
Fetching additional content from URLs...
Researching: Airbnb regulatory challenges issues faced by Airbnb
Found 10 total results
After deduplication: 10 results
Fetching additional content from URLs...
Researching: Airbnb pricing model challenges competition regulation discrimination trust adaptation profitability
Researching: Airbnb business challenges regulation trust safety discrimination quality competition housing affordability community impact international markets
Found 12 total results
After deduplication: 8 results
Fetching additional content from URLs...
Found 10 total results
After deduplication: 8 results
Fetching additional content from URLs...
Error fetching content from https://www.researchgate.net/publication/342489642_The_evolutio

In [6]:

m.chat("What Challenges Does BMW Face?",web=False,rag=True,tokens=100,use_gpt=False,use_sub_gpt=True,iters=3)



100%|██████████| 4/4 [00:00<00:00, 1127.12it/s]


'BMW faces various challenges due to its involvement with environmental regulations, particularly those related to waste management and producer responsibility policies.\n\nSome key issues include:\n\n*   Economic costs associated with implementing these measures.\n    *   Involvement from dismantle workers who may resist changes brought by stricter rules on vehicle scrapping.\n\n\n\nThese factors highlight some potential difficulties facing companies involved in automotive manufacturing when dealing with regulatory requirements around responsible product lifecycle practices.\n\n\nHowever it is worth noting there have been efforts made towards sustainability within industries including car'

In [7]:
m.conscious

[{'role': 'user', 'content': 'What Challenges Does Airbnb Face?'},
 {'role': 'assistant',
  'content': "Airbnb faces several challenges including:\n\n1\\. **Regulatory Issues**: Many cities around the world impose regulations or restrictions on short-term rentals which can impact revenue for host.\n\n2\\. \\*\\*Pricing Model\\***\\: The company's algorithm-based price setting system has raised concerns about fairness as some users report lower rates from Black owners compared to white ones despite having identical listings.\n\n\n\n3\\. Social Media Backlash : Negative publicity surrounding racism within its community led by prominent figures like Brian Anderson, former CEO. This backlash resulted in significant financial losses due largely attributed towards increased regulatory scrutiny.\n\n4\\. Competition From Other Platforms :\n\n5\\. Data Security Concerns\n\n\n\nTo address these challenges, I'd advise Chris Chezsky (CEO) to conduct an investigation into his Pricing Algorithm focu

In [8]:
m.subconscious

[[{'user': 'Airbnb faces several significant challenges as it continues to operate and grow globally:\n\n1. **Regulations and Legal Issues:** Many cities and countries are introducing stricter regulations on short-term rentals to address concerns about housing affordability, neighborhood disruption, and tax collection. Navigating these changing laws can be complex and costly.\n\n2. **Trust and Safety:** Ensuring the safety of both guests and hosts is crucial. Airbnb must work constantly to prevent scams, address property damage, and resolve disputes fairly. Building and maintaining trust in the platform is an ongoing effort.\n\n3. **Discrimination and Inclusivity:** Reports of bias or discrimination on the platform have pressured Airbnb to create policies and tools that promote inclusivity and prevent unfair treatment of users.\n\n4. **Quality Control:** Since listings are individually managed by hosts rather than by Airbnb directly, maintaining consistent quality and guest experiences